In [1]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import skimage
from skimage import io, transform
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms


In [2]:
EPOCHS = 30
data_dir = "/content/drive/MyDrive/chest_xray"
TEST = 'test'
TRAIN = 'train'
VAL = 'val'

In [3]:
def data_transforms(phase):
    if phase == TRAIN:
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
    elif phase == VAL:
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
    elif phase == TEST:
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
    return transform

In [4]:
def run_with_transform(transform, device):
    print(f"Testing transform: {transform}")
    transform = transforms.Compose([
        transform,
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), transform)
                      for x in [TRAIN, VAL, TEST]}

    dataloaders = {TRAIN: torch.utils.data.DataLoader(image_datasets[TRAIN], batch_size=4, shuffle=True),
                   VAL: torch.utils.data.DataLoader(image_datasets[VAL], batch_size=1, shuffle=True),
                   TEST: torch.utils.data.DataLoader(image_datasets[TEST], batch_size=1, shuffle=True)}

    dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL]}
    classes = image_datasets[TRAIN].classes
    class_names = image_datasets[TRAIN].classes

    model_pre = models.vgg16(pretrained=True)
    for param in model_pre.features.parameters():
        param.required_grad = False

    num_features = model_pre.classifier[6].in_features
    features = list(model_pre.classifier.children())[:-1]
    features.extend([nn.Linear(num_features, len(class_names))])
    model_pre.classifier = nn.Sequential(*features)

    model_pre = model_pre.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model_pre.parameters(), lr=0.001, momentum=0.9, weight_decay=0.01)
    exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [5]:
# Run the parametric sweep for each transform
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transforms_to_test = [
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    # Add more transforms here as needed
]

for transform in transforms_to_test:
    run_with_transform(transform, device)

Testing transform: Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:04<00:00, 118MB/s]


Testing transform: CenterCrop(size=(224, 224))
Testing transform: RandomHorizontalFlip(p=0.5)
Testing transform: RandomRotation(degrees=[-30.0, 30.0], interpolation=nearest, expand=False, fill=0)
Testing transform: ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=(-0.2, 0.2))
Testing transform: RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1))


In [9]:

TRAIN = 'train'
VAL = 'val'
TEST = 'test'

def data_transforms(phase, resize_size, crop_size, normalize_mean, normalize_std):
    if phase == TRAIN or phase == VAL or phase == TEST:
        transform = transforms.Compose([
            transforms.Resize(resize_size),
            transforms.CenterCrop(crop_size),
            transforms.ToTensor(),
            transforms.Normalize(normalize_mean, normalize_std),
        ])
    else:
        raise ValueError("Invalid phase")

    return transform

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Define parameters for the sweep
resize_sizes = [256, 512]  # Vary resize size
crop_sizes = [224, 448]    # Vary crop size
normalize_means = [[0.485, 0.456, 0.406], [0.5, 0.5, 0.5]]  # Vary mean for normalization
normalize_stds = [[0.229, 0.224, 0.225], [0.1, 0.1, 0.1]]    # Vary std for normalization

# Sweep through parameters
for resize_size in resize_sizes:
    for crop_size in crop_sizes:
        for normalize_mean in normalize_means:
            for normalize_std in normalize_stds:
                transform = data_transforms(phase=TRAIN,
                                             resize_size=resize_size,
                                             crop_size=crop_size,
                                             normalize_mean=normalize_mean,
                                             normalize_std=normalize_std)
                print("Resize Size:", resize_size)
                print("Crop Size:", crop_size)
                print("Normalize Mean:", normalize_mean)
                print("Normalize Std:", normalize_std)
                print("Transform:", transform)
                print("\n")

cuda:0
Resize Size: 256
Crop Size: 224
Normalize Mean: [0.485, 0.456, 0.406]
Normalize Std: [0.229, 0.224, 0.225]
Transform: Compose(
    Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


Resize Size: 256
Crop Size: 224
Normalize Mean: [0.485, 0.456, 0.406]
Normalize Std: [0.1, 0.1, 0.1]
Transform: Compose(
    Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.1, 0.1, 0.1])
)


Resize Size: 256
Crop Size: 224
Normalize Mean: [0.5, 0.5, 0.5]
Normalize Std: [0.229, 0.224, 0.225]
Transform: Compose(
    Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.229, 0.224, 0.225])
)


Resize Size: 256
Crop Size: 224
Normalize Mean: 